## 1) Install JAX, FLAX, Optax and TFDS. 

==> **Flax** can use any dataset loading pipeline (Tensorflow Datasets or Pytorch DataLoader).

In [1]:
!pip install --upgrade -q pip jax jaxlib flax optax tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.7 MB/s eta 0:00

##2) Import Libraries

In [2]:
import jax

#JAX Numpy lets you run code on GPUs & TPUs
import jax.numpy as jnp 

from flax import linen as nn      #The Linen API
from flax.training import train_state

#The Optax gradient processing and optimization library 
import optax 

import numpy as np 

In [3]:
import tensorflow
import tensorflow_datasets as tfds 

# We can load the built in datasets from this function
from tensorflow.keras.datasets import mnist

def get_datasets():
  ds_builder = tfds.builder('mnist')
  ds_builder.download_and_prepare()
  # Split into training/test sets
  train_ds = tfds.as_numpy(ds_builder.as_dataset(split='train', batch_size=-1))
  test_ds = tfds.as_numpy(ds_builder.as_dataset(split='test', batch_size=-1))
  # Convert to floating-points
  train_ds['image'] = jnp.float32(train_ds['image']) / 255.0
  test_ds['image'] = jnp.float32(test_ds['image']) / 255.0
  return train_ds, test_ds

## 3) Build a Model 
Build a convolutional neural network with the Flax Linen API by subclassing `flax.linen.Module`. 

In [4]:
class CNN(nn.Module):

  @nn.compact
  # Provide a constructor to register a new parameter 
  # and return its initial value
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1)) # Flatten
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)    # There are 10 classes in MNIST
    return x

## 4) Create a metrics function 
For loss and accuracy metrics, create a separate function:
- **Optax** has a built-in softmax cross-entropy loss (`optax.softmax_cross_entropy`).
- The labels can be **one-hot** encoded with `jax.nn.one_hot`.

In [5]:
def compute_metrics(logits, labels):
  loss = jnp.mean(optax.softmax_cross_entropy(logits, jax.nn.one_hot(labels, num_classes=10)))
  accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
  metrics = {
      'loss': loss,
      'accuracy': accuracy
  }
  return metrics

In [6]:
@jax.jit
def train_step(state, batch):
  def loss_fn(params):
    logits = CNN().apply({'params': params}, batch['image'])
    loss = jnp.mean(optax.softmax_cross_entropy(
        logits=logits, 
        labels=jax.nn.one_hot(batch['label'], num_classes=10)))
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (_, logits), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  metrics = compute_metrics(logits, batch['label'])
  return state, metrics

In [7]:
@jax.jit
def eval_step(params, batch):
  logits = CNN().apply({'params': params}, batch['image'])
  return compute_metrics(logits, batch['label'])

In [8]:
def train_epoch(state, train_ds, batch_size, epoch, rng):
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size

  perms = jax.random.permutation(rng, len(train_ds['image']))
  perms = perms[:steps_per_epoch * batch_size]  # Skip an incomplete batch
  perms = perms.reshape((steps_per_epoch, batch_size))

  batch_metrics = []

  for perm in perms:
    batch = {k: v[perm, ...] for k, v in train_ds.items()}
    state, metrics = train_step(state, batch)
    batch_metrics.append(metrics)

  training_batch_metrics = jax.device_get(batch_metrics)
  training_epoch_metrics = {
      k: np.mean([metrics[k] for metrics in training_batch_metrics])
      for k in training_batch_metrics[0]}

  print('Training - epoch: %d, loss: %.4f, accuracy: %.2f' % (epoch, training_epoch_metrics['loss'], training_epoch_metrics['accuracy'] * 100))

  return state, training_epoch_metrics

In [9]:
def eval_model(model, test_ds):
  metrics = eval_step(model, test_ds)
  metrics = jax.device_get(metrics)
  eval_summary = jax.tree_map(lambda x: x.item(), metrics)
  return eval_summary['loss'], eval_summary['accuracy']

In [10]:
rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)

In [11]:
cnn = CNN()
params = cnn.init(init_rng, jnp.ones([1, 28, 28, 1]))['params']

In [12]:
learning_rate = 0.001
tx = optax.sgd(learning_rate=learning_rate)

In [13]:
state = train_state.TrainState.create(apply_fn=cnn.apply, params=params, tx=tx)

In [14]:
num_epochs = 25
batch_size = 128

In [15]:
train_ds, test_ds = get_datasets()

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [16]:
import time

start = time.time() 

for epoch in range(1, num_epochs + 1):
  # Use a separate PRNG key to permute image data during shuffling
  rng, input_rng = jax.random.split(rng)
  # Run an optimization step over a training batch
  state, train_metrics = train_epoch(state, train_ds, batch_size, epoch, input_rng)
  # Evaluate on the test set after each training epoch
  test_loss, test_accuracy = eval_model(state.params, test_ds)
  print('Testing - epoch: %d, loss: %.2f, accuracy: %.2f' % (epoch, test_loss, test_accuracy * 100))

print('Finished Training')
print("Total time: ", time.time() - start, "seconds") 

Training - epoch: 1, loss: 2.2437, accuracy: 32.41
Testing - epoch: 1, loss: 2.16, accuracy: 60.28
Training - epoch: 2, loss: 2.0582, accuracy: 66.50
Testing - epoch: 2, loss: 1.91, accuracy: 73.00
Training - epoch: 3, loss: 1.6968, accuracy: 72.72
Testing - epoch: 3, loss: 1.42, accuracy: 77.04
Training - epoch: 4, loss: 1.1834, accuracy: 77.76
Testing - epoch: 4, loss: 0.93, accuracy: 82.32
Training - epoch: 5, loss: 0.8114, accuracy: 82.19
Testing - epoch: 5, loss: 0.67, accuracy: 85.12
Training - epoch: 6, loss: 0.6258, accuracy: 84.82
Testing - epoch: 6, loss: 0.54, accuracy: 86.83
Training - epoch: 7, loss: 0.5316, accuracy: 86.22
Testing - epoch: 7, loss: 0.48, accuracy: 87.73
Training - epoch: 8, loss: 0.4776, accuracy: 87.25
Testing - epoch: 8, loss: 0.43, accuracy: 88.54
Training - epoch: 9, loss: 0.4425, accuracy: 87.91
Testing - epoch: 9, loss: 0.41, accuracy: 88.87
Training - epoch: 10, loss: 0.4176, accuracy: 88.40
Testing - epoch: 10, loss: 0.39, accuracy: 89.33
Training